In [1]:
from DataClass import *
from PlotClass import *
from DataFuncs import *
from CONSTANTS import  *


error while signin
you are using nologin method, data you access may be limited


In [2]:
reader = CsvReader(csvs_path)

NameError: name 'files' is not defined